<a href="https://colab.research.google.com/github/AilingLiu/Inferential_Statistics/blob/master/Inferential_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook summarizes the testing methods from <b>Inferential Statitistics </b> course taught by <u>University of Amsterdam</u> on [Coursera](https://www.coursera.org/learn/inferential-statistics). The course had taught how to conduct statistical test usng R. Here, I am using Python to do the test. All the formulas used in this document can be found [here](https://github.com/AilingLiu/Inferential_Statistics/blob/master/FormulasTables.pdf).

In [0]:
import numpy as np
import pandas as pd
import scipy.stats as st

# Compare Two Groups

<b>Construct Hypotheses</b>

When we are testing between two competing hypotheses, a null hypothesis $H_0$ and an alternative hypothesis $H_1$, we generally assume that the null hypothesis is true unless the data shows a strong indication that this is not the case. 

By doing hypotheses testing, we <u>test the probability of finding a sample statistic given that the null hypothesis is true</u>. If the null hypothesis is true, the difference between a sample statistics and the population parameter is <b>due to sampling error</b>, that is, fluctuations in the sample from the population. However, **if the probability of finding a sample statistic as extreme as ours under the null hypothesis is very small, we generally reject the null hypothesis**.

> Test your understanding:


1.   Imagine we have found a p value of 0.30 called p1 and another p value of 0.02 called p2, do these p values indicate strong evidence or weak evidence in favour of the null hypothesis? 
>> Answer: p1 indicates strong evidence in favour of the null hypothesis; p2 indicates weak evidence in favour of the null hypothesis.
2.   What does a p value of 0.20 mean?
>> Answer: A p value of 0.20 means that there's a probability of 20% of obtaining a similar result or more extreme given that the null hypothesis is true



## Z test to compare two proportions from independent samples</b>


We usually calculate two things:

1.   The difference between two sample proportions
2.   The standard error

> Example
<br>In this exercise we have a sample of 100 males with a proportion of left wing voters of 0.6 and a sample of 150 females with a proportion of left-wing voters of 0.42. 

In [5]:
nmale=100
nfemale=150
malep=0.6
femalep=0.42

#pooled proportion
poolp=(nmale*malep+nfemale*femalep)/(nmale+nfemale)

#standard error under the null hypothesis
se=np.sqrt(poolp*(1-poolp)*(1/nmale + 1/nfemale))

#z calculated value
z_val = (malep - femalep)/se

#corresponding p value
p_val = (1-st.norm.cdf(z_val))*2

sig = 0.05
if p_val <=sig:
  conclusion='Rejected'
else:
  conclusion='Not enough evidence to reject'

print(f'Calculated Z value: {z_val:.4f}\nPvalue is: {p_val:.4f} \nConclusion on Null Hypothesis given {sig} significance level: {conclusion}')

Calculated Z value: 2.7889
Pvalue is: 0.0053 
Conclusion on Null Hypothesis given 0.05 significance level: Rejected


Another way to conduct the test is to get the confidence interval of the difference from the two proportions. If 0 (null hypothesis) falls inside the interval, we will reject null hypotheseis. We need two parameters to conduct this test:

1.   The z score corresponding to the selected confidence level: $(1-conf)/2$.
2.   The standarad error for the difference between two proportions

In [6]:
#z score under given confidence level
sig=0.01
z_score = np.abs(st.norm.ppf(sig/2))

# standard error for the difference
sep=np.sqrt(malep*(1-malep)/nmale + femalep*(1-femalep)/nfemale)

#lower bound of confidence interval
lb = (malep-femalep) - z_score*sep
#upper bound of confidence interval
ub = (malep-femalep) + z_score*sep

print(f'{(1-sig)*100} percent confidence interval:\n[{lb:.4f}, {ub:.4f}]')

99.0 percent confidence interval:
[0.0166, 0.3434]


There are differences and these differences are significant because the 99% confidence interval does not contain 0.

The equivalent z test for two independent proportions is [proportions_ztest](https://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.proportions_ztest.html).

In [7]:
#equivalent in python

from statsmodels.stats.proportion import proportions_ztest
x_success = np.array([nmale*malep, nfemale*femalep])
n_total = np.array([nmale, nfemale])
z_val, p_val = proportions_ztest(count=x_success, nobs=n_total, alternative='two-sided')

#make a function to give conclusion directly based on pvalue and significance level
def testeval(sig, pval):
  
  """
  Conclusion of rejection status on null hypothesis given significance level
  and the pvalue corresponding to calculated test statistic.

  Parameters
  ----------
  sig: float
    Significance level. Governs the chance of a false positive.
      A significance level of 0.05 means that there is a 5% chance of
      a false positive.
  
  pval: float
    Calculated p value. The probability of obtaining a similar results
    or more extreme given null hypothesis is true.
  
  Returns:
  --------
  result: string
    Conclusion of rejection status on null hypothesis.
  """

  if pval <=sig:
    result = 'Rejected'
  else:
    result = 'Not enough evidence to reject'
  return result

siglevel = 0.05
conclusion = testeval(siglevel, p_val)
print(f'Calculated Z value: {z_val:.4f}\nPvalue is: {p_val:.4f} \nConclusion on Null Hypothesis given {siglevel} significance level: {conclusion}\n')

siglevel = 0.01
conclusion = testeval(siglevel, p_val)
print(f'Calculated Z value: {z_val:.4f}\nPvalue is: {p_val:.4f} \nConclusion on Null Hypothesis given {siglevel} significance level: {conclusion}\n')

def prop_confint_2ind(count, nobs, alpha=0.05):
  
  """
  A/B test for two proportions;
  given a success a trial size of group A and B compute
  its confidence interval;
  resulting confidence interval matches R's prop.test function

  Parameters
  ----------
  count: array
      Number of successes in each group

  nobs: array
      Size, or number of observations in each group

  alpha : float, default 0.05
      Significance level. Governs the chance of a false positive.
      A significance level of 0.05 means that there is a 5% chance of
      a false positive. In other words, our confidence level is
      1 - 0.05 = 0.95

  Returns
  -------
  prop_diff : float
      Difference between the two proportion

  confint : 1d ndarray
      Confidence interval of the two proportion test
  """  

  a_success, b_success = count[0], count[1]
  a_size, b_size = nobs[0], nobs[1]
  a_prop, b_prop = a_success/a_size, b_success/b_size
  prop_diff = a_prop-b_prop

  #z score under given confidence level
  z_score = np.abs(st.norm.ppf(alpha/2))

  # standard error for the difference
  sep=np.sqrt(a_prop*(1-a_prop)/a_size + b_prop*(1-b_prop)/b_size)

  #lower bound of confidence interval
  lb = prop_diff - z_score*sep
  #upper bound of confidence interval
  ub = prop_diff + z_score*sep
  return prop_diff, [lb, ub]

sig=0.01
diff, [lowerb, upperb] = prop_confint_2ind(count=x_success, nobs=n_total, alpha=sig)
print(f'{(1-sig)*100} percent confidence interval:\n[{lowerb:.4f}, {upperb:.4f}]')

Calculated Z value: 2.7889
Pvalue is: 0.0053 
Conclusion on Null Hypothesis given 0.05 significance level: Rejected

Calculated Z value: 2.7889
Pvalue is: 0.0053 
Conclusion on Null Hypothesis given 0.01 significance level: Rejected

99.0 percent confidence interval:
[0.0166, 0.3434]


## T test to compare compare two means from independent samples

we usually calculate 2 other things first

1.   The difference between two independent sample means
2.   The standard error of the difference between two independent sample means

> Example
<br>In this exercise we have a sample of 100 males that do sports on average 4.2 hours per week and a sample of 150 females that do sports on average 5.8 hours per week. 
*  Case a: the population variances are unequal in two groups
*  Case b: the populatin variances are equal in two groups

In [15]:
#Case a: the population variances are unequal
nmale=100
malemean=4.2
stdmale=2.3
nfemale=150
femalemean=5.8
stdfemale=3.1

#standard eror for the difference between two means
se=np.sqrt(stdmale**2/nmale+stdfemale**2/nfemale)

#mean difference
diff=malemean-femalemean

#t value
t_val=diff/se

#degree of freedom
df=se**2/((1/(nmale-1)*(stdmale**2/nmale)**2)+(1/(nfemale-1)*(stdfemale**2/nfemale)**2))

#calculate the p value
pval=(1-st.t.cdf(np.abs(t_val), df))*2
siglevel=0.01
conclusion=testeval(siglevel, pval)
print(f'Calculated T value: {t_val:.4f}\nPvalue is: {pval:.4f} \nConclusion on Null Hypothesis given {siglevel} significance level: {conclusion}\n')

# calculate the 99% confidence interval
t_score=np.abs(st.t.ppf(siglevel/2, df))
lb = diff-t_score*(se)
ub = diff+t_score*(se)
print(f'{(1-siglevel)*100} percent confidence interval:\n[{lb:.4f}, {ub:.4f}]')


Calculated T value: -4.6783
Pvalue is: 0.0000 
Conclusion on Null Hypothesis given 0.01 significance level: Rejected

99.0 percent confidence interval:
[-2.4817, -0.7183]


In [13]:
#Case b: the population variances are equal
nmale=100
malemean=4.2
nfemale=150
femalemean=5.8
std=2.8

#mean difference
diff=malemean-femalemean

#pooled standard deviation
s=np.sqrt(((nmale-1)*std**2 + (nfemale-1)*std**2)/(nmale-1+nfemale-1))

#standard eror for the difference between two means
se=s*np.sqrt(1/nmale+1/nfemale)

#t value
t_val=diff/se

#degree of freedom
df=nmale+nfemale-2

#calculate the p value
pval=(1-st.t.cdf(np.abs(t_val), df))*2
siglevel=0.01
conclusion=testeval(siglevel, pval)
print(f'Calculated T value: {t_val:.4f}\nPvalue is: {pval:.4f} \nConclusion on Null Hypothesis given {siglevel} significance level: {conclusion}\n')

# calculate the 99% confidence interval
t_score=np.abs(st.t.ppf(siglevel/2, df))
lb = diff-t_score*(se)
ub = diff+t_score*(se)
print(f'{(1-siglevel)*100} percent confidence interval:\n[{lb:.4f}, {ub:.4f}]')


Calculated T value: -4.4263
Pvalue is: 0.0000 
Conclusion on Null Hypothesis given 0.01 significance level: Rejected

99.0 percent confidence interval:
[-2.5383, -0.6617]


Equivalent t test for two independent is [ttest_ind](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_ind.html) from scipy or [ttest_ind](https://www.statsmodels.org/stable/generated/statsmodels.stats.weightstats.ttest_ind.html) from statsmodels. Both methods take data points as array directly, without specifically giving mean, standard deviation, or size.

In [35]:
from statsmodels.stats.weightstats import ttest_ind

#generate random data with mean, std, size as above sample.
## equal variance
rvmale=np.random.normal(loc=malemean, scale=std, size=nmale)
rvmale_fix = (rvmale - np.mean(rvmale)) * (std / np.std(rvmale)) + malemean #fix mean problem
rvfemale=np.random.normal(loc=femalemean, scale=std, size=nfemale)
rvfemale_fix = (rvfemale - np.mean(rvfemale)) * (std / np.std(rvfemale)) + femalemean #fix mean problem

t_val, pval, df=ttest_ind(rvmale_fix, rvfemale_fix, alternative='two-sided', usevar='pooled', value=0)
conclusion=testeval(0.01, pval)
print(f'Calculated T value: {t_val:.4f}\nPvalue is: {pval:.4f} \nConclusion on Null Hypothesis given {0.01} significance level: {conclusion}\n')

Calculated T value: -4.4085
Pvalue is: 0.0000 
Conclusion on Null Hypothesis given 0.01 significance level: Rejected



In [41]:
## unequal variance
rvmale=np.random.normal(loc=malemean, scale=stdmale, size=nmale)
rvmale_fix = (rvmale - np.mean(rvmale)) * (stdmale / np.std(rvmale)) + malemean #fix mean problem
rvfemale=np.random.normal(loc=femalemean, scale=stdfemale, size=nfemale)
rvfemale_fix = (rvfemale - np.mean(rvfemale)) * (stdfemale / np.std(rvfemale)) + femalemean #fix mean problem

t_val, pval, df=ttest_ind(rvmale_fix, rvfemale_fix, alternative='two-sided', usevar='unequal', value=0)
conclusion=testeval(0.01, pval)
print(f'Calculated T value: {t_val:.4f}\nPvalue is: {pval:.4f} \nConclusion on Null Hypothesis given {0.01} significance level: {conclusion}\n')

Calculated T value: -4.6591
Pvalue is: 0.0000 
Conclusion on Null Hypothesis given 0.01 significance level: Rejected



How to interpret the result?

Given that the null hypothesis is true, there is a probability of 0.000005 (5.21345e-06) of obtaining a result equally or more extreme. We are 99% confident that the population difference in hours of sport per week between males and females is between -2.4817 and -0.7183 hours per week.

## Comparing two proportions for paired sample - McNemar's Test

Working with dependent data, such as twins, couples, same subject from different time, we will need to use different methods from above.

> Example
<br> Our research question here is whether there is a difference between the proportion of surveyed individuals that approve of the European union and the proportion of their partners that approve of the European union. What would be a good pair of hypotheses?
<br>Answer
<br>$H_0$: The proportion of EU approval is not different in surveyed individuals and their partners. $H_1$: The proportion of EU approval is different in surveyed individuals and their partners

In [65]:
import pandas as pd

col_index=pd.MultiIndex.from_tuples([('Partner Approves of the EU', 'Yes'), ('Partner Approves of the EU', 'No')])
row_index=pd.MultiIndex.from_tuples([('Survey Individuals that approve of the EU', 'Yes'),('Survey Individuals that approve of the EU', 'No')])
survey = pd.DataFrame(np.array([[150, 50], [35, 100]]), index=row_index, columns=col_index)
survey['row_totals'] = survey.sum(axis=1)
s=survey.sum(axis=0)
s.name='column_totals'
survey = survey.append(s)
display(survey)

Partner Approves of the EU  ... row_totals
                                                                        Yes  ...           
(Survey Individuals that approve of the EU, Yes)                        150  ...        200
(Survey Individuals that approve of the EU, No)                          35  ...        135
column_totals                                                           185  ...        335

[3 rows x 3 columns]

In [67]:
#calculate z value
z_val=(50-35)/np.sqrt(50+35)

#get pvalue
pval=(1-st.norm.cdf(np.abs(z_val)))*2
siglevel=0.05
conclusion=testeval(siglevel, pval)
print(f'Calculated Z value: {z_val:.4f}\nPvalue is: {pval:.4f} \nConclusion on Null Hypothesis given {siglevel} significance level: {conclusion}\n')

Calculated Z value: 1.6270
Pvalue is: 0.1037 
Conclusion on Null Hypothesis given 0.05 significance level: Not enough evidence to reject



The equivalent [mcnemar's test](http://www.statsmodels.org/dev/generated/statsmodels.stats.contingency_tables.mcnemar.html) in statsmodelss.

In [85]:
from statsmodels.stats.contingency_tables import mcnemar
result = mcnemar(survey.iloc[:2, :2].to_numpy(), exact=False, correction=False)
print(result)

pvalue      0.1037416782365415
statistic   2.6470588235294117


## Compare two means for paired samples

> Example
<br>An example when we would do this is if we would want to know the effectiveness of a diet on people's weight. Our research question here is whether the diet that we have invented leads to a reduction in weight. As such our research question is directional. What would be a good set of hypotheses?
<br>Answer: 
<br>$H_0$: There is no difference in people's weight before and after the diet. $H_1$: There is a reduction in weight after taking the diet.

In [91]:
#generate data
pre_weight=np.random.normal(loc=81.53587, scale=8.113578, size=100)
pre_weight_fix=(pre_weight-np.mean(pre_weight))*(8.113578/np.std(pre_weight))+81.5358

post_weight=np.random.normal(loc=78.20945, scale=9.223542, size=100)
post_weight_fix=(post_weight-np.mean(post_weight))*(9.223542/np.std(pre_weight))+78.20945

# get the difference of the two means
diff = pre_weight_fix.mean()-post_weight_fix.mean()

#standard deviation of the differences
stddiff = np.std(pre_weight_fix-post_weight_fix)

#standard error of the difference
se=stddiff/np.sqrt(100)

tval=diff/se 
pval=(1-st.t.cdf(np.abs(tval), 100-1))*2
siglevel=0.05
conclusion=testeval(siglevel, pval)
print(f'Calculated t value: {tval:.4f}\nPvalue is: {pval:.4f} \nConclusion on Null Hypothesis given {siglevel} significance level: {conclusion}\n')

Calculated t value: 2.3788
Pvalue is: 0.0193 
Conclusion on Null Hypothesis given 0.05 significance level: Rejected

